# Parametric Complexity

## 1. Introduction

The *parametric complexity* of a model family $\mathcal{M}$ is a measure of how much information it tends to *memorise* when fitted using a particular estimator. In the case of MLE, it measures how susceptible the *most likley* model in the family is to overfitting. Then it can be used to make sound comparisons between the *probabilities* of different model families as follows. Let



### $$L_{\text{nml}}(\mathcal{M}|D) = L_{M^*}(D) + L_\mathcal{M}(\mathcal{M})$$

where:

- $\mathcal{M}$ is a model family amoung some set of candiates $\mathcal{M}_1,\dots,\mathcal{M}_n$.

- $D$ is a dataset from some family $\mathcal{D}$.

- $L_{\text{nml}}$ is the *posterior* length of $\mathcal{M}$ given $D$.

- $L_{M^*}$ is the *likelihood* length of $D$ under the MLE estimate $M^*$. 

- $L_\mathcal{M}$ is the *prior* length of $\mathcal{M}$, i.e. its parametric complexity.


Then the *most probable* model family is the one with shortest posterior length. To compute this length, we first need to compute $L_{M^*}$ and $L_\mathcal{M}$. Luckily, this is rather straightforward and avoids the arbitrariness of crude MDL.


To compute $L_{M^*}$, we simply find the MLE esimate $M^* \in \mathcal{M}$ and then compute $L_{M^*}(D) = -\log_2 p_{M^*}(D)$. In other words, we calculate how much the MLE can compress the dataset.

To compute $L_{\mathcal{M}}$, we use the following algorithm:

1. Pick a dataset from $\mathcal{D}$, such that each data point is chosen uniformly at random. Neccessarily, the chance that at least $k$ bits can be removed from this dataset is exponentially small in $k$.

2. Compute $L_{M^*}$ for this dataset and subtract it from the dataset's original size in bits. Since the dataset cannot feasibly be compressed, this measures how much information is simply *memorised* in $M^*$.

3. Repeat steps 1-2 several times to obtain an average for how many bits are memorised.

4. Return this average as the parametric complexity of $\mathcal{M}$.

> Note 1: for step 2, we assume that the number of bits removed from the dataset is sufficiently large, such that the probability that those bits *were memorised* is very close to 1. For example, if 50 bits are removed, the probability that they were memorised is $1 - 2^{-50} =  0.9999999999999991$. It's exeedingly likely that those bits were simply transfered to the model rather than genuinly compressed.
>
> Note 2: this algorithm estimates parametric complexity w.r.t. a particular dataset family $\mathcal{D}$. This is a specification for how many features and examples are in the dataset. It's important that $D \in \mathcal{D}$ when we calculate $L_M^*$, i.e. it must have the same number of features and examples. 

In [ ]:
from typing import List

In [ ]:
class Sampler:

    def sample(self, length: int) -> List:
        raise NotImplementedError()

In [ ]:
class Model:

    def fit(self):
        raise NotImplementedError()

    def score(self):
        raise NotImplementedError()

In [ ]:
def binaryClassifierComplexity(model: Model, sampler: Sampler, trials: int = 1000, samples: int = 1000, return_scores=False):
    """
    Computes parametric complexity for biniary classifiers.
    """

    scores: List[float] = []

    for i in range(trials):
        
        dataset = sampler.sample(samples)
        hypothesis = model.fit(*dataset)

        probits = model.predict_proba(dataset[0])
        probits_0 = probits[:,0]
        probits_1 = probits[:,1]

        probits = (probits_0 * (dataset[1] == 0)) + (probits_1 * (dataset[1] == 1))
        logits = -np.log2(probits)

        compressed_size = np.sum(logits)
        scores.append(samples - compressed_size)
    
    if return_scores:
        return scores
    
    return np.mean(scores)

In [ ]:
from sklearn.linear_model import LogisticRegression
import numpy as np

In [ ]:
class LogisticRegressionSampler(Sampler):

    def sample(self, length):

        X = np.random.rand(length, 1)
        y = (np.random.rand(length) < 0.5).astype(int)

        return (X, y)

In [ ]:
sampler = LogisticRegressionSampler()
model = LogisticRegression()

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier

model_10 = MLPClassifier(hidden_layer_sizes = (10,), max_iter=8000)
model_100 = MLPClassifier(hidden_layer_sizes = (100,), max_iter=3000)
model_100_10 = MLPClassifier(hidden_layer_sizes = (100,10), max_iter=10_000)
model_100_100 = MLPClassifier(hidden_layer_sizes = (100,100), max_iter=10_000)
model_500_500 = MLPClassifier(hidden_layer_sizes = (500,500), max_iter=10_000)
model_tree = DecisionTreeClassifier(max_depth=5)

In [ ]:
binaryClassifierComplexity(model_10, sampler, trials=20, samples=100)

0.8962468390922013

In [ ]:
binaryClassifierComplexity(model_100, sampler, trials=20, samples=100)

1.3222902848864622

In [ ]:
binaryClassifierComplexity(model_100_10, sampler, trials=20, samples=100)

1.432378231222114

In [ ]:
binaryClassifierComplexity(model_tree, sampler, trials=20, samples=100)

34.374701795783224

In [ ]:
ds = sampler.sample(10)

In [ ]:
model_tree.fit(*ds)

DecisionTreeClassifier()

In [ ]:
model_tree.score(*ds)

1.0

In [ ]:
ds[0] 

array([[0.39394108],
       [0.46045414],
       [0.09321846],
       [0.0281672 ],
       [0.79684971],
       [0.26325962],
       [0.7879565 ],
       [0.35116863],
       [0.02461956],
       [0.57953887]])